<h1 align='center'>Aplicando o aprendizado de máquina à análise de sentimentos</h1>

<p align='center'><img src= https://cibersistemas.pt/wp-content/uploads/2020/09/wall-5.jpeg width='500'></p>


Na era moderna da internet e das mídias sociais, as opiniões, avaliações e recomendações das pessoas se tornaram um recurso valioso para a ciência política e os negócios. Graças às tecnologias modernas, agora podemos coletar e analisar esses dados com mais eficiência. Neste capítulo, vamos nos aprofundar em um subcampo do processamento de linguagem natural (NLP) chamado análise de sentimentos e aprender a usar algoritmos de aprendizado de máquina para classificar documentos com base em sua polaridade: a atitude do escritor. Em particular, vamos trabalhar com um conjunto de dados de 50.000 críticas de filmes do Internet Movie Database (IMDb) e construir um preditor que possa distinguir entre críticas positivas e negativas.

## Preparando os dados de revisão de filme do IMDb para processamento de texto
Como mencionado, a análise de sentimentos, às vezes também chamada de **mineração de opinião**, é uma subdisciplina popular do campo mais amplo da *NLP*; preocupa-se em analisar a polaridade dos documentos. Uma tarefa popular na análise de sentimentos é a classificação de documentos com base nas opiniões ou emoções expressas dos autores em relação a um tópico específico.

In [12]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


In [13]:
df.shape

(50000, 2)

### Apresentando o modelo de *bag-of-words*

Apresentaremos o modelo *bag-of-words*, que nos permite representar o texto como vetores numéricos de recursos. A ideia por trás do *bag-of-words* é bastante simples e pode ser resumido da seguinte forma:

1. Criamos um vocabulário de tokens exclusivos — por exemplo, palavras — de todo o conjunto de documentos.
2. Construímos um vetor de características de cada documento que contém as contagens de quantas vezes cada palavra ocorre no documento em particular.

Como as palavras únicas em cada documento representam apenas um pequeno subconjunto de todas as palavras no vocabulário do *bag-of-words*, os vetores de recursos consistirão principalmente em zeros, e é por isso que os chamamos de esparsos. Não se preocupe se isso soar muito abstrato; mais pra frente, veremos o passo a passo do processo de criação de um modelo simples de um *bag-of-words*.

### Transformando palavras em vetores de recursos
Para construir um modelo *bag-of-words* baseado na contagem de palavras nos respectivos documentos, podemos usar a classe *CountVectorizer* implementada no *scikit-learn*. Como você verá no código a seguir, o *CountVectorizer* pega uma matriz de dados de texto, que podem ser documentos ou frases, e constrói o modelo *bag-of-words*:

In [14]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

# Agora vamos imprimir o conteúdo do vocabulário para entender melhor os conceitos subjacentes:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


Como você pode ver ao executar o comando anterior, o vocabulário é armazenado em um dicionário *Python* que mapeia as palavras exclusivas para índices inteiros. Em seguida, vamos imprimir os vetores de recursos que acabamos de criar:

In [15]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Cada posição de índice nos vetores de recursos mostrados aqui corresponde aos valores inteiros armazenados como itens de dicionário no vocabulário *CountVectorizer*. Por exemplo, a primeira *feature* na posição de índice 0 se assemelha à contagem da palavra 'and', que ocorre apenas no último documento, e a palavra 'is', na posição de índice 1 (a segunda feição nos vetores do documento), ocorre nas três frases. Esses valores nos vetores de características também são chamados de frequências de termos brutos: tf(t, d)— número de vezes que um termo, *t*, ocorre em um documento, **d*. Deve-se notar que, no modelo *bag-of-words*, a ordem das palavras ou dos termos em uma frase ou documento não importa.

A ordem em que as frequências dos termos aparecem no vetor de características é derivada dos índices de vocabulário, que geralmente são atribuídos em ordem alfabética.

<blockquote>
<h4><i>Modelos de N-gram</i></h4>
<p alig='justify'>A sequência de itens no modelo de <i>bag-of-words</i> que acabamos de criar também é chamada de modelo de <i>1-gram</i> ou <i>unigram</i> – cada item ou <i>token</i> no vocabulário representa <u>uma única palavra</u>. De maneira geral, as sequências contíguas de itens no <i>NLP</i>  – palavras, letras ou símbolos – também são chamadas de <i>n-grams</i>. A escolha do número, <i>n</i>, no modelo <i>n-gram</i> depende da aplicação particular; por exemplo, um estudo de <i>Ioannis Kanaris</i> e outros revelou que <i>n-grams</i> de <b>tamanho 3 e 4</b> rendem bons desempenhos na filtragem anti-spam de mensagens de e-mail.</p>

<p alig='justify'>Para resumir o conceito da representação <i>n-gram</i>, as representações de <i>1-gram</i> e <i>2-gram</i> do nosso primeiro documento "the sun is shining" seriam construídas da seguinte forma:
<ul>
<li> <i>1-gram</i>: "the", "sun", "is", "shining"</li>
<li> <i>2-gram</i>: "the sun", "sun is", "is shining"</li>
</ul>
<p alig='justify'>A classe <i>CountVectorizer</i> no <i>scikit-learn</i> nos permite usar diferentes modelos de <i>n-gram</i> por meio de seu parâmetro <i>ngram_range</i>. Embora uma representação de <i>1-gram</i> seja usada por padrão, podemos alternar para uma representação de <i>2-gram</i> inicializando uma nova instância <i>CountVectorizer</i> com <b><i>ngram_range=(2,2)</i></b>.
</blockquote>

### Avaliando a relevância da palavra por meio da frequência do documento inversa da frequência do termo
Quando estamos analisando dados de texto, geralmente encontramos palavras que ocorrem em vários documentos de ambas as classes. Essas palavras que ocorrem com frequência geralmente não contêm informações úteis ou discriminatórias. Agora, falaremos sobre uma técnica útil chamada o **termo de frequência e frequência documento inverso (tf-idf)**, que pode ser usado para reduzir o peso dessas palavras que ocorrem com frequência nos vetores de recursos.

A biblioteca *scikit-learn* implementa ainda outro transformador, a classe `TfidfTransformer`, que pega as frequências de termos brutos da classe *CountVectorizer* como entrada e as transforma em `tf-idfs`:

In [16]:
np.set_printoptions(precision=2)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


Como você viu anteriomente, a palavra 'is' teve a maior frequência de termos no terceiro documento, sendo a palavra mais frequente. No entanto, após transformar o mesmo vetor de características em *tf-idfs*, a palavra 'is' agora está associada a um *tf-idf* relativamente pequeno (0,45) no terceiro documento, pois também está presente no primeiro e no segundo documento e, portanto, é improvável que contenha qualquer informação discriminatória útil. No entanto, se tivéssemos calculado manualmente os *tf-idfs* dos termos individuais em nossos vetores de recursos, teríamos notado que o `TfidfTransformer` calcula o *tf-idfs* de forma ligeiramente diferente em comparação com as equações padrão dos livros didáticos que definimos anteriormente.

### Limpando dados de texto

Aprendemos sobre o modelo *bag-of-words*, frequências de termos e *tf-idfs*. No entanto, o primeiro passo importante antes de construirmos nosso modelo de *bag-of-words* é limpar os dados de texto removendo todos os caracteres indesejados.

In [17]:
# Para ilustrar por que isso é importante, vamos exibir os últimos 500 caracteres
# do primeiro documento no conjunto de dados de revisão de filme reordenado

df.loc[0,'review'][-500:]

"film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Simon Yam was my favorite character here and sort of anchors the picture.<br /><br />Election was quite the award winner at last year's Hong Kong Film Awards, winning for best actor (Tony Leung), best picture, best director (Johnny To, who did Heroic Trio!!), and best screenplay. It also had nominations for cinematography, editing, film score (which I loved), and three more acting performances (including Yam)."

Como você pode ver aqui, o texto contém marcação *HTML*, bem como pontuação e outros caracteres que não são letras. Embora a marcação *HTML* não contenha muitas semânticas úteis, os sinais de pontuação podem representar informações úteis e adicionais em determinados contextos de *NLP*. No entanto, para simplificar, agora removeremos todos os sinais de pontuação, exceto os caracteres de *emoticon*, como *:)*, pois certamente são úteis para análise de sentimentos. Para realizar essa tarefa, usaremos a biblioteca *mlxtend*, conforme mostrado aqui:

In [18]:
from mlxtend.text import tokenizer_words_and_emoticons

tokenizer_words_and_emoticons("</a>This :) is :( a test :-)!")

['this', 'is', 'a', 'test', ':)', ':(', ':-)']

<blockquote>
<h5>Lidando com a capitalização da palavra</h5>
<p align="justify">No contexto desta análise, assumimos que a capitalização de uma palavra – por exemplo, se ela aparece no início de uma frase – não contém informações semanticamente relevantes. No entanto, observe que há exceções; por exemplo, removemos a notação de nomes próprios. Mas, novamente, no contexto desta análise, é uma suposição simplificadora que a letra maiúscula não contém informações relevantes para a análise de sentimentos.</p>
</blockquote>

Eventualmente, adicionamos os *emoticons* armazenados temporariamente ao final da *string *do documento processado. Além disso, removemos o caractere de nariz (- em :-)) dos *emoticons* para consistência.

Embora a adição dos caracteres de *emoticon* ao final das sequências de documentos limpados possa não parecer a abordagem mais elegante, devemos observar que a ordem das palavras não importa em nosso modelo de *bag-of-words* se nosso vocabulário consiste em apenas *tokens* de uma palavra.

### Processando documentos em tokens
Depois de preparar com sucesso o conjunto de dados de resenhas de filmes, agora precisamos pensar em como dividir os corpora de texto em elementos individuais. Uma maneira de tokenizar documentos é dividi-los em palavras individuais, dividindo os documentos limpos em seus caracteres de espaço em branco.

No contexto de tokenização, outra técnica útil é retirar o radical das palavras (*word stemming*), que é o processo de transformar uma palavra em sua forma raiz. Ele nos permite mapear palavras relacionadas ao mesmo radical. O algoritmo de *lematização* original foi desenvolvido por *Martin F. Porter*, em 1979, e, portanto, é conhecido como algoritmo de *lematização* de *Porter*. O *Natural Language Toolkit* (NLTK, http://www.nltk.org) para *Python* implementa o algoritmo de determinação de *Porter*, que usaremos no código a seguir:

In [19]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

[porter.stem(word) for word in tokenizer_words_and_emoticons('runners like running and thus they run')]

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

Usando o *PorterStemmer* do pacote *nltk*, modificamos nossa função *tokenizer* para reduzir as palavras à sua forma raiz, o que foi ilustrado pelo exemplo simples acima em que a palavra *'running'* foi derivada para sua forma raiz *'run'*.

<blockquote>
<h5>Algoritmos de derivação</h5>
<p>O algoritmo de <i>stemming de Porter</i> é provavelmente o algoritmo de <i>stemming</i> mais antigo e simples. Outros algoritmos de lematização populares incluem o mais recente lematizador <i>Snowball</i> (<i>Porter2</i> ou lematizador inglês) e o lematizador <i>Lancaster</i> (lematizador Paice/Husk). Enquanto as raízes <i>Snowball</i> e <i>Lancaster</i> são mais rápidas do que a original <i>Porter</i>, a <i>Lancaster</i> também é notória por ser mais agressiva que a <i>Porter</i>. Esses algoritmos alternativos também estão disponíveis através do pacote NLTK (http://www.nltk.org/api/nltk.stem.html).

<p>Enquanto a <i>Stemming</i> criar palavras não reais, como 'thu' (de 'thus' ("assim", do inglês)), como mostrado no exemplo anterior, uma técnica chamada lematização visa obter as formas canônicas (gramaticalmente corretas) de palavras individuais - as chamadas <i>lemmas</i>. No entanto, a lematização é <b>computacionalmente mais difícil e cara</b> em comparação com a <i>stemming</i> e, na prática, observa-se que a <i>stemming</i> e a lematização tem pouco impacto no desempenho da classificação de texto.</p>
</blockquote>

In [20]:
tokenizer_words_and_emoticons(df.loc[0,'review'][-500:])

['film',
 'ends',
 'at',
 'the',
 '65',
 'or',
 '70',
 'minute',
 'mark',
 'there',
 'are',
 'still',
 'a',
 'couple',
 'big',
 'surprises',
 'waiting',
 'simon',
 'yam',
 'was',
 'my',
 'favorite',
 'character',
 'here',
 'and',
 'sort',
 'of',
 'anchors',
 'the',
 'picture',
 'election',
 'was',
 'quite',
 'the',
 'award',
 'winner',
 'at',
 'last',
 'year',
 's',
 'hong',
 'kong',
 'film',
 'awards',
 'winning',
 'for',
 'best',
 'actor',
 'tony',
 'leung',
 'best',
 'picture',
 'best',
 'director',
 'johnny',
 'to',
 'who',
 'did',
 'heroic',
 'trio',
 'and',
 'best',
 'screenplay',
 'it',
 'also',
 'had',
 'nominations',
 'for',
 'cinematography',
 'editing',
 'film',
 'score',
 'which',
 'i',
 'loved',
 'and',
 'three',
 'more',
 'acting',
 'performances',
 'including',
 'yam']

Por fim, como usaremos os dados de texto limpos repetidamente durante as próximas seções, vamos agora aplicar nossa função de pré-processador a todas as resenhas de filmes em nosso *DataFrame*:

In [21]:
# df['review'] = df['review'].apply(tokenizer_words_and_emoticons)
df

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1
3,"Dear Readers,<br /><br />The final battle betw...",1
4,I have seen The Perfect Son about three times....,1
...,...,...
49995,"If you see the title ""2069 A Sex Odyssey"" in t...",0
49996,There were but two reasons for me to see this ...,0
49997,i saw this movie the first seconds the voice o...,1
49998,This is another one of those 'humans vs insect...,0


Antes de passarmos para frente, onde treinaremos um modelo de aprendizado de máquina usando o modelo *bag-of-words*, vamos falar brevemente sobre outro tópico útil chamado **remoção de palavras irrelevantes**. *Stop-words* são simplesmente aquelas palavras que são extremamente comuns em todos os tipos de textos e provavelmente não contêm nenhuma (ou apenas uma pequena) informação útil que possa ser usada para distinguir entre diferentes classes de documentos. Exemplos de *Stop-words* são *is*, *and*, *has* e *like*.

A remoção de *Stop-words* pode ser útil se estivermos trabalhando com frequências de termos brutos ou normalizados em vez de *tf-idfs*, que já estão reduzindo o peso de palavras que ocorrem com frequência. Para remover *Stop-words* das resenhas de filmes, usaremos o conjunto de 127 *Stop-words*, em inglês, que está disponível na biblioteca *NLTK*, que pode ser obtido chamando a função *nltk.download*:

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[porter.stem(word) for word in tokenizer_words_and_emoticons('a runner likes running and runs a lot')[-10:] if word not in stop]

['runner', 'like', 'run', 'run', 'lot']

### Treinando um modelo de regressão logística para classificação de documentos

Treinaremos um modelo de regressão logística para classificar as resenhas de filmes em críticas positivas e negativas com base no modelo *bag-of-words*. Primeiro, dividiremos o D*ataFrame* de documentos de texto limpos em **2.500 documentos** para treinamento e **2.500 documentos** para teste:

In [24]:
X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values

Em seguida, usaremos um objeto *GridSearchCV* para encontrar o conjunto ideal de parâmetros para nosso modelo de regressão logística usando validação cruzada estratificada de 5 vezes:

In [25]:
import nltk
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# Instanciando Algoritmo PorterStemmer
porter = PorterStemmer()

# Definindo funções Tokenizer e Tokenizer Porter 
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

##########################################
# Separando os valores X_train y-train 
##########################################
X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values


# Definindo os parâmentros do GridSearch
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer_porter, tokenizer],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0]},
              ]

# Instanciando o tfidf
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

# Criando um pipeline
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

# Instanciando o GridSearchCV
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

# Treinando o Modelo GridSearchCV 
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                          

In [26]:
# Verificando o melhor conjunto de Parâmentros:
print(f"Melhor Conjunto de Parâmentros:\n {gs_lr_tfidf.best_params_}")
print(f"\nAcurácia CV: {gs_lr_tfidf.best_score_:.2%}")

Melhor Conjunto de Parâmentros:
 {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000002A282A43940>}

Acurácia CV: 83.05%


In [27]:
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values
clf = gs_lr_tfidf.best_estimator_
print(f"Acurácia no Conjunto de Teste: {clf.score(X_test, y_test):.2%}")

Acurácia no Conjunto de Teste: 83.59%


Os resultados revelam que nosso modelo de aprendizado de máquina pode prever se uma crítica de filme é positiva ou negativa com 83,59% de precisão.

### Trabalhando com dados maiores – algoritmos online e aprendizado *out-of-core*
Construir os vetores de *features* para o conjunto de dados de revisão de 50.000 filmes durante um *Grid Search* pode ser **computacionalmente muito dispendioso**. Em muitos aplicativos do mundo real, não é incomum trabalhar com conjuntos de dados ainda maiores que podem exceder a memória do nosso computador. Como nem todos têm acesso a instalações de supercomputadores, agora aplicaremos uma técnica chamada aprendizado *out-of-core*, que nos permite trabalhar com conjuntos de dados tão grandes ajustando o classificador de forma incremental em lotes menores de um conjunto de dados.

O conceito de *Gradient Descend* estocástico, já conhecido, é um algoritmo de otimização que atualiza os pesos do modelo usando um exemplo por vez. Agora, usaremos a função `partial_fit` do *SGDClassifier* do *scikit-learn* para transmitir os documentos diretamente de nossa unidade local e treinar um modelo de regressão logística usando pequenos mini-lotes de documentos.

Primeiro, vamos definir uma função *tokenizer* que limpa os dados de texto não processados do arquivo *movie_data.csv* que construímos no início e os separa em *tokens* de palavras, enquanto remove as *stop_words*:

In [28]:
# Usando a mlxtend
# Primeiro, vamos definir uma função tokenizer que limpa os dados de texto não processados do arquivo movie_data.csv que 
# e os separa em tokens de palavras enquanto remove stop_words:

def tokenizer(text):
    tokenizer = tokenizer_words_and_emoticons(text)
    return [w for w in tokenizer if w not in stop]


# Em seguida, definiremos uma função geradora, stream_docs, que lê e retorna um documento por vez
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label


# Agora vamos definir uma função, get_minibatch, que pegará um fluxo de documentos da função stream_docs
# e retornará um determinado número de documentos especificados pelo parâmetro size:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [29]:
next(stream_docs(path='movie_data.csv'))

('"Election is a Chinese mob movie, or triads in this case. Every two years an election is held to decide on a new leader, and at first it seems a toss up between Big D (Tony Leung Ka Fai, or as I know him, ""The Other Tony Leung"") and Lok (Simon Yam, who was Judge in Full Contact!). Though once Lok wins, Big D refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader. Unlike any other Asian film I watch featuring gangsters, this one is not an action movie. It has its bloody moments, when necessary, as in Goodfellas, but it\'s basically just a really effective drama. There are a lot of characters, which is really hard to keep track of, but I think that plays into the craziness of it all a bit. A 100-year-old baton, which is the symbol of power I mentioned before, changes hands several times before things settle down. And though it may appear that the film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Si

Infelizmente, não podemos usar `CountVectorizer` para aprendizado *out-of_core*, pois requer manter o vocabulário completo na memória. Além disso, o `TfidfVectorizer` precisa manter todos os vetores de recursos do conjunto de dados de treinamento na memória para calcular as frequências inversas do documento. No entanto, outro vetorizador útil para processamento de texto implementado no *scikit-learn* é o `HashingVectorizer`. `HashingVectorizer` é independente de dados e faz uso do truque de *hash* por meio da função *MurmurHash3 de 32 bits de Austin Appleby*(https://sites.google.com/site/murmurhash/).

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1)


doc_stream = stream_docs(path='movie_data.csv')

Usando o código anterior, inicializamos o `HashingVectorizer` com nossa função tokenizer e definimos o número de recursos para 2**21. Além disso, reinicializamos um classificador de regressão logística definindo o parâmetro de perda de SGDClassifier como `log`. Observe que, ao escolher um grande número de recursos no `HashingVectorizer`, reduzimos a chance de causar colisões de *hash*, mas também aumentamos o número de coeficientes em nosso modelo de regressão logística.

Agora vem a parte realmente interessante – tendo configurado todas as funções complementares, podemos iniciar o aprendizado *out-of-core* usando o seguinte código:

In [31]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Inicializamos o objeto da barra de progresso com 45 iterações e, no *loop for* a seguir, iteramos mais de 45 minilotes de documentos onde cada minilote consiste em <u>1.000 documentos</u>. Concluído o processo de aprendizado incremental, usaremos os últimos <u>5.000 documentos</u> para avaliar o desempenho do nosso modelo:

In [32]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f"Acurácia: {clf.score(X_test, y_test):.2%}")

Acurácia: 86.56%


Como você pode ver, a precisão do modelo é de aproximadamente *87%*, um pouco abaixo da precisão que alcançamos anteriormente usando o *Grid Search* para ajuste de hiperparâmetros. No entanto, o aprendizado *out-of-core* é muito eficiente em termos de memória e levou <u>menos de um minuto</u> para ser concluído. Finalmente, podemos usar os últimos <b>5.000 documentos</b> para atualizar nosso modelo:

In [33]:
clf = clf.partial_fit(X_test, y_test)

<blockquote>
<h4> Modelo <i>word2vec</i></h4> 
O algoritmo <i>word2vec</i> é um algoritmo de aprendizado <b>não supervisionado</b> baseado em redes neurais que tenta aprender automaticamente a relação entre as palavras. A ideia por trás do <i>word2vec</i> é colocar palavras que tenham significados semelhantes em <i>clusters</i> semelhantes e, por meio de espaçamento vetorial inteligente, o modelo pode reproduzir certas palavras usando matemática vetorial simples, por exemplo, <u>king – man + women = queen</u>.
</blockquote>


### Modelagem de tópicos com Alocação de *Dirichlet* Latente (*LDA*)

A modelagem de tópicos descreve a ampla tarefa de atribuir tópicos a documentos de texto não rotulados. Por exemplo, uma aplicação típica seria a categorização de documentos em um grande corpus de texto de artigos de um jornal. Em aplicações de modelagem de tópicos, procuramos atribuir rótulos de categoria a esses artigos, por exemplo, esportes, finanças, notícias mundiais, política, notícias locais e assim por diante. Assim, no contexto das amplas categorias de aprendizado de máquina, podemos considerar a modelagem de tópicos como uma tarefa de agrupamento, uma subcategoria de aprendizado não supervisionado.

Agora, discutiremos uma técnica popular para modelagem de tópicos chamada Alocação de *Dirichlet* Latente (*LDA*). No entanto, observe que, embora a alocação de *Dirichlet* latente seja frequentemente abreviada como *LDA*, ela não deve ser confundida com a <u>Análise Discriminante Linear (LDA)</u>, uma técnica de redução de dimensionalidade supervisionada.

### Decompondo documentos de texto com LDA

Como a matemática por trás do *LDA* é bastante complexa e requer conhecimento sobre inferência bayesiana, abordaremos esse tópico da perspectiva de um praticante e interpretaremos o *LDA* usando termos leigos. *LDA* é um modelo probabilístico generativo que tenta encontrar grupos de palavras que aparecem frequentemente juntas em diferentes documentos. Essas palavras que aparecem com frequência representam nossos tópicos, assumindo que cada documento é uma mistura de palavras diferentes. A entrada para um *LDA* é o modelo de *bag-of-words* que discutimos anteriormente. Dada uma matriz *bag-of-words* como entrada, o *LDA* a decompõe em duas novas matrizes:
* Uma matriz de documento para tópico
* Uma matriz palavra a tópico

O *LDA* decompõe a matriz *bag-of-words* de tal forma que, se multiplicarmos essas duas matrizes, conseguiremos reproduzir a entrada, a matriz *bag-of-words*, com <u>o menor erro possível</u>. Na prática, estamos interessados ​​naqueles tópicos que o *LDA* encontrou na matriz *bag-of-words*. A única desvantagem pode ser que devemos definir o número de tópicos de antemão - o número de tópicos é um hiperparâmetro de *LDA* que deve ser especificado manualmente.

### *LDA* com *scikit-learn*
Usaremos a classe `LatentDirichletAllocation` implementada no *scikit-learn* para decompor o conjunto de dados de resenhas de filmes e categorizá-lo em diferentes tópicos. No exemplo a seguir, restringiremos a análise a 10 tópicos diferentes, mas podemos experimentar os hiperparâmetros do algoritmo para explorar mais os tópicos que podem ser encontrados neste conjunto de dados.


In [34]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')

# Em seguida, usaremos o já familiar CountVectorizer para criar a matriz bag-of-words como entrada para o LDA.
# Por conveniência, usaremos a biblioteca de stopwords "em inglês" integrada do scikit-learn via stop_words='english':

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

Observe que definimos **a frequência máxima de palavras** do documento a serem consideradas como **10 por cento** (`max_df=.1`) para excluir palavras que ocorrem com muita frequência nos documentos. A lógica por trás da remoção de palavras que ocorrem com frequência é que essas podem ser palavras comuns que aparecem em todos os documentos que são, portanto, menos prováveis de serem associadas a uma categoria de tópico específica de um determinado documento.

Além disso, limitamos o número de palavras a serem consideradas às **5.000 palavras mais frequentes** (`max_features=5000`), para limitar a dimensionalidade desse conjunto de dados para melhorar a inferência realizada pelo *LDA*. No entanto, `max_df=.1` e `max_features=5000` são valores de hiperparâmetros escolhidos arbitrariamente, e podemos ajustá-los ao comparar os resultados.

In [36]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, 
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

Ao definir `learning_method='batch'`, deixamos o estimador *lda* fazer sua estimativa com base em todos os dados de treinamento disponíveis (a matriz *bag-of-words*) em uma iteração, que é mais lenta do que o <u>método alternativo de aprendizado 'online'</u>, mas pode levar a resultados mais precisos (configurar `learning_method= 'online'` é análogo ao aprendizado online ou em minilote).

Após ajustar o *LDA*, agora temos acesso ao atributo `components_` da instância *lda*, que armazena uma matriz contendo a importância da palavra (**no caso, 5000**) para cada um dos **10 tópicos** em ordem crescente:

In [37]:
lda.components_.shape

(10, 5000)

Para analisar os resultados, vamos imprimir **as cinco palavras** mais importantes para cada **um dos 10 tópicos**. Observe que os valores de importância da palavra são classificados <u>em ordem crescente</u>. Assim, para imprimir as cinco principais palavras, precisamos ordenar o array de tópicos na ordem inversa:

In [38]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
    for i in topic.argsort()
    [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music history
Topic 4:
human audience cinema art feel
Topic 5:
police guy car dead murder
Topic 6:
horror house sex blood gore
Topic 7:
role performance comedy actor performances
Topic 8:
series episode episodes tv season
Topic 9:
book version original effects read
Topic 10:
action fight guy fun guys


Com base na leitura das cinco palavras mais importantes para cada tópico, você pode adivinhar que a *LDA* identificou os seguintes tópicos:
1. Filmes geralmente ruins (não é realmente uma categoria de tópico)
2. Filmes sobre famílias
3. Filmes de guerra
4. Filmes de arte
5. Filmes policiais
6. Filmes de terror
7. Críticas de filmes de comédia
8. Filmes de alguma forma relacionados a programas de TV
9. Filmes baseados em livros
10. Filmes de ação

Para confirmar que as categorias fazem sentido com base nos comentários, vamos plotar três filmes da categoria de filmes de terror (filmes de terror pertencem à categoria 6 na posição 5 do índice):

In [40]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings tha ...

Horror movie #2:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...

Horror movie #3:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...


Usando o exemplo de código anterior, imprimimos os primeiros <u>300 caracteres dos três principais filmes de terror</u>. As resenhas – mesmo que não saibamos a qual filme exatamente elas pertencem – soam como resenhas de filmes de terror, Embora a base tenha repetido o Filme 2 e 3.